In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_netflix = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
print(df_netflix.shape)
df_netflix.head()

In [ ]:
plt.figure(figsize=(10,7))
sns.set(style='darkgrid')
ax = sns.countplot(x='type', data = df_netflix)

In [ ]:
df_netflix['country'].unique()

In [ ]:
#We can see that countries have been coupled. So, we will separate them and add the counts.

import itertools
list_country = [x.split(', ') for x in df_netflix.dropna(subset=['country'])['country'].tolist()]
list_country = list(itertools.chain(*list_country))

from collections import Counter
df_netflix_country_count = pd.DataFrame(Counter(list_country).most_common()[:10],
                                        columns= ['Country', 'Count'])

In [ ]:
plt.figure(figsize=(12,10))
#sns.set(style="darkgrid")
ax = sns.barplot(y="Country", x='Count', data=df_netflix_country_count)

In [ ]:
plt.figure(figsize=(12,10))
ax = sns.countplot(y='release_year', data=df_netflix, order=df_netflix['release_year'].value_counts().index[0:15])

### Preprocessing of the data for RS

#### Retaining only relavant columns

In [ ]:
df_netflix.drop(columns= ['director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'type'],
               inplace= True)
df_netflix.head()

### Removing Punctuations and Stopwords

In [ ]:
from nltk.tokenize import word_tokenize

df_netflix['title_list'] = df_netflix['title'].str.lower()
df_netflix['listed_in'] = df_netflix['listed_in'].str.lower()
df_netflix['description'] = df_netflix['description'].str.lower()

df_netflix['title_list'] = df_netflix['title_list'].apply(word_tokenize)
df_netflix['listed_in'] = df_netflix['listed_in'].apply(word_tokenize)
df_netflix['description'] = df_netflix['description'].apply(word_tokenize)

In [ ]:
from nltk.corpus import stopwords
from string import punctuation

list_stopwords = set(stopwords.words('english') + list(punctuation))
df_netflix['title_list'] = df_netflix['title_list'].apply(lambda x : [word for word in x if word not in list_stopwords])
df_netflix['listed_in'] = df_netflix['listed_in'].apply(lambda x : [word for word in x if word not in list_stopwords])
df_netflix['description'] = df_netflix['description'].apply(lambda x : [word for word in x if word not in list_stopwords])

In [ ]:
import string

df_netflix['description'] = df_netflix['description'].apply(lambda x : [word.translate(str.maketrans('', '', string.punctuation)) for word in x])

df_netflix['description'] = df_netflix['description'].apply(lambda x : [word for word in x if len(word) > 0])

In [ ]:
df_netflix['title_list'] = df_netflix['title_list'].apply(lambda x : list(set(x)))
df_netflix['listed_in'] = df_netflix['listed_in'].apply(lambda x : list(set(x)))
df_netflix['description'] = df_netflix['description'].apply(lambda x : list(set(x)))

In [ ]:
df_netflix['description']

## Download pretrained word2vec model from Google

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!gunzip GoogleNews-vectors-negative300.bin.gz

In [ ]:
import gensim

wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

### Find Similarities Among Shows using Title, Genres, Description

In [ ]:
matrix_netflix_vocab = []
for list_ in df_netflix.to_numpy():
    list_[2] = [word for word in list_[2] if word in wv.vocab]
    list_[3] = [word for word in list_[3] if word in wv.vocab]
    list_[4] = [word for word in list_[4] if word in wv.vocab]
    matrix_netflix_vocab.append(list_)
df_netflix_vocab = pd.DataFrame(matrix_netflix_vocab, columns= df_netflix.columns)

In [ ]:
from tqdm import tqdm

def recommendation(title):
    
    matrix_netflix_title_vocab = []
    for list_ in df_netflix[df_netflix['title'] == title].to_numpy():
        list_[2] = [word for word in list_[2] if word in wv.vocab]
        list_[3] = [word for word in list_[3] if word in wv.vocab]
        list_[4] = [word for word in list_[4] if word in wv.vocab]
        matrix_netflix_title_vocab.append(list_)
        
    matrix_similarity = []
    pbar = tqdm(matrix_netflix_vocab)
    for list1 in pbar:
        for list2 in matrix_netflix_title_vocab:
            score_catg = wv.n_similarity(list1[2], list2[2])
            score_desc = wv.n_similarity(list1[3], list2[3])
            try:
                score_title = wv.n_similarity(list1[4], list2[4])/2
            except:
                score_title = 0
            if ((list1[1] != list2[1]) & (score_catg > 0.85)):
                matrix_similarity.append([list1[1], list2[1], score_title, score_catg, score_desc])
        pbar.update()
    pbar.close()
    df_netflix_similarity = pd.DataFrame(matrix_similarity,columns = ['recommendation','title','score_title', 'score_category', 'score_description'])
    
    df_netflix_similarity['final_score'] = df_netflix_similarity['score_title'] + df_netflix_similarity['score_category'] + df_netflix_similarity['score_description']
    
    return (df_netflix_similarity.sort_values(by=['final_score', 'score_category', 'score_description', 'score_title'], ascending=False).head(10))


In [ ]:
recommendation('Avengers: Infinity War')

In [ ]:
recommendation('Black Panther')

In [ ]:
recommendation('Friends')


In [ ]:
recommendation('Gilmore Girls')

In [ ]:
recommendation('13 Reasons Why')